In [1]:
import mne
import numpy as np
from glob import glob
import csv
import h5py
from tqdm.notebook import tqdm
import sys
sys.path.append('../../preprocessing/utils/')
import strf

In [2]:
# Change these path for running the notebook locally
eeg_data_path = '/path/to/dataset/' # downloadable from OSF: https://doi.org/10.17605/OSF.IO/FNRD9
git_path  = '/path/to/git/speaker_induced_suppression_EEG/'
# Where the output of train_linear_model.ipynb is saved. Run that first if you haven't already.
# Alternatively, model h5 are available in the OSF repository, in folder `linear_encoding_model_results`.
h5_path = '/path/to/h5'

In [3]:
picks = ['F1','Fz','F2','FC1','FCz','FC2','C1','Cz','C2']
tmin,tmax = -1.5, 3.5
baseline=(None,0)
reject_thresh = 10 # SD
exclude = 'OP0020'
subjs = np.sort([s[-6:] for s in glob(f'{git_path}eventfiles/*') if 'OP0' in s and exclude not in s])

## Load ERP data

In [4]:
def interpolate_bads(subj,block,raw,git_path,eeg_data_path):
    '''
    CCA .vhdr files do not include bad channels, so this func interpolates them.
    The end result is each subject has a 64 channel raw arrray, which makes plotting easier
    Filter the data before running this!
    '''
    blockid = subj + '_' + block
    nsamps = len(raw)
    info = mne.io.read_raw_brainvision(f'{eeg_data_path}OP0001/OP0001_B1/OP0001_B1_cca.vhdr',
                                           preload=False,verbose=False).info
    ch_names = info['ch_names']
    bads = [c for c in ch_names if c not in raw.info['ch_names']]
    if len(bads) > 0:
        new_data = []
        for ch in ch_names:
            if ch in bads:
                print('Interpolating', ch)
                new_data.append(np.zeros((1,nsamps)))
            else:
                new_data.append(raw.get_data(picks=ch)[0])
        raw = mne.io.RawArray(np.vstack(new_data),info)
        raw.info['bads'] = bads
        raw.interpolate_bads()
    else:
        print(subj, 'has no bads.')
    return raw

In [5]:
raws, spkr_epochs, mic_epochs, spkr_resp, mic_resp = dict(), dict(), dict(), dict(), dict()
conditions = ['el','sh','all']
channels = ['spkr', 'mic']
conditions
subj_bar = tqdm(subjs)
for s in subj_bar:
    spkr_epochs[s], mic_epochs[s], spkr_resp[s], mic_resp[s] = dict(), dict(), dict(), dict()
    if s != 'OP0002':
        block = 'B1'
    else:
        block = 'B2'
    blockid = s + '_' + block
    raw_path = f'{eeg_data_path}{s}/{blockid}/{blockid}_cca.vhdr'
    raw = mne.io.read_raw_brainvision(raw_path,preload=True,verbose=False)
    raw.filter(l_freq=1,h_freq=30,verbose=False)
    raws[s] = interpolate_bads(s,block,raw,git_path,eeg_data_path)
    for ch in channels:
        spkr_epochs[s][ch], mic_epochs[s][ch], spkr_resp[s][ch], mic_resp[s][ch] = dict(), dict(), dict(), dict()
        for co in conditions:
            subj_bar.set_description(f'Epoching {s} {ch} {co}')
            if s in ['OP0015','OP0016']: # mtpl blocks
                events = []
                for b in ['B1','B2']: 
                    event_file = f'{git_path}eventfiles/{s}/{s}_{b}/{s}_{b}_{ch}_sn_{co}.txt'
                    if b == 'B2':
                        b2_fpath = raw_path = f'{eeg_data_path}{s}/{s}_B1/{s}_B1_downsampled.vhdr'
                        b2_raw = mne.io.read_raw_brainvision(b2_fpath,preload=True,verbose=False)
                        block_shift = b2_raw.last_samp
                    else:
                        block_shift = 0
                    with open(event_file, 'r') as my_csv:
                        csvReader = csv.reader(my_csv, delimiter='\t')
                        for row in csvReader:
                            onset = int((float(row[0])*128)+block_shift)
                            offset = int((float(row[1])*128)+block_shift)
                            sn_id = int(row[2])
                            events.append([onset,offset,sn_id])
                events = np.array(events,dtype=int)
            else:
                event_file = f'{git_path}eventfiles/{s}/{blockid}/{blockid}_{ch}_sn_{co}.txt'
                events = []
                with open(event_file, 'r') as my_csv:
                    csvReader = csv.reader(my_csv, delimiter='\t')
                    for row in csvReader:
                        onset = int((float(row[0])*128))
                        offset = int((float(row[1])*128))
                        sn_id = int(row[2])
                        events.append([onset,offset,sn_id])
                events = np.array(events,dtype=int)
            reject = mne.Epochs(raws[s], events, tmin=tmin, tmax=tmax,reject=None,
                           baseline=baseline,verbose=False)
            reject = reject.get_data(picks=picks)
            reject = dict(eeg=np.std(reject)*(reject_thresh*2))
            epochs = mne.Epochs(raws[s],events,tmin=tmin,tmax=tmax,reject=reject,
                                   baseline=baseline,verbose=False)
            if ch == 'spkr':
                spkr_epochs[s][ch][co] = epochs
                spkr_resp[s][ch][co] = epochs.get_data(picks=picks)
            if ch == 'mic':
                mic_epochs[s][ch][co] = epochs
                mic_resp[s][ch][co] = epochs.get_data(picks=picks)

  0%|          | 0/20 [00:00<?, ?it/s]

OP0001 has no bads.
Using data from preloaded Raw for 202 events and 641 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 202 events and 641 original time points ...
    Rejecting  epoch based on EEG : ['PO3']
    Rejecting  epoch based on EEG : ['PO3']
    Rejecting  epoch based on EEG : ['Pz']
    Rejecting  epoch based on EEG : ['PO3']
    Rejecting  epoch based on EEG : ['PO3']
    Rejecting  epoch based on EEG : ['PO3']
    Rejecting  epoch based on EEG : ['PO3', 'POz']
    Rejecting  epoch based on EEG : ['T7']
8 bad epochs dropped
Using data from preloaded Raw for 202 events and 641 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 202 events and 641 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 404 events and 641 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 404 events and 641 original time points ...
    Rejecting  epoch based on EEG : ['PO3']
    Re

0 bad epochs dropped
Using data from preloaded Raw for 398 events and 641 original time points ...
    Rejecting  epoch based on EEG : ['P6']
    Rejecting  epoch based on EEG : ['P6']
    Rejecting  epoch based on EEG : ['P6']
3 bad epochs dropped
Using data from preloaded Raw for 200 events and 641 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 200 events and 641 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 200 events and 641 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 200 events and 641 original time points ...
    Rejecting  epoch based on EEG : ['POz']
    Rejecting  epoch based on EEG : ['POz']
2 bad epochs dropped
Using data from preloaded Raw for 400 events and 641 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 400 events and 641 original time points ...
    Rejecting  epoch based on EEG : ['POz']
    Rejecting  epoch based on EEG : ['POz']
2 

    Rejecting  epoch based on EEG : ['Fp1', 'Fz', 'F3', 'F7', 'FT10', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FC3', 'FCz', 'F6', 'F2', 'AF4']
    Rejecting  epoch based on EEG : ['F8', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'F7', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['AF7']
    Rejecting  epoch based on EEG : ['AF7']
12 bad epochs dropped
OP0006 has no bads.
Using data from preloaded Raw for 149 events and 641 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 149 events and 641 original time points ...
    Rejecting  epoch based on EEG : ['O2', 'P4', 'PO4']
    Rejecting  epoch based on EEG : ['O2', 'P4', 'PO3', 'PO4', 'P6']
    Rejecting  epoch based on EEG : ['O2', 'PO3', 'PO4']
    Rejecting  epoch based on EEG : ['O2', 'P4', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2']
    Rejecting  epoch based on EEG : ['O2', 'P4', 'PO4', 'PO8', 'P6']
    Rejecting  epoch based on EEG : ['O2', 'P4', 'POz', 'PO4', 'P6']
    Reje

    Rejecting  epoch based on EEG : ['Fp1', 'F3', 'O2', 'P4', 'PO3', 'POz', 'PO4', 'PO8', 'P6']
    Rejecting  epoch based on EEG : ['Pz', 'O2', 'P4', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2']
    Rejecting  epoch based on EEG : ['Fp1', 'FT9', 'Pz', 'O2', 'P4', 'P8', 'Fp2', 'AF7', 'P5', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2']
    Rejecting  epoch based on EEG : ['O2', 'P4', 'P8', 'PO3', 'PO4', 'PO8', 'P6']
    Rejecting  epoch based on EEG : ['O2', 'PO4']
    Rejecting  epoch based on EEG : ['Pz', 'P3', 'O2', 'P4', 'P1', 'P5', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2']
    Rejecting  epoch based on EEG : ['O2', 'P4', 'PO4', 'P6']
    Rejecting  epoch based on EEG : ['O2', 'P4', 'PO3', 'POz', 'PO4', 'P6']
    Rejecting  epoch based on EEG : ['O2', 'P4', 'PO3', 'PO4', 'PO8', 'P6']
    Rejecting  epoch based on EEG : ['O2', 'PO4', 'P6']
    Rejecting  epoch based on EEG : ['O2', 'P4', 'PO3', 'POz', 'PO4', 'PO8', 'P6']
    Rejecting  epoch based on EEG : ['O2', 'P4', 'PO4', 'P6']
    Rejecting 

...

Using data from preloaded Raw for 141 events and 641 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 141 events and 641 original time points ...
    Rejecting  epoch based on EEG : ['F3', 'FT9', 'FC5', 'AF7', 'AF3', 'AFz', 'F1', 'F5']
    Rejecting  epoch based on EEG : ['Fz', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'Cz', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'FCz', 'C1', 'C5', 'C2', 'FC4', 'F6', 'F2', 'AF4', 'AF8']
2 bad epochs dropped
Using data from preloaded Raw for 298 events and 641 original time points ...


/tmp/ipykernel_38160/52083338.py:27: RuntimeWarning: Not setting positions of 4 misc channels found in montage:
['spkr', 'mic', 'vEOG', 'hEOG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  b2_raw = mne.io.read_raw_brainvision(b2_fpath,preload=True,verbose=False)
/tmp/ipykernel_38160/52083338.py:27: RuntimeWarning: No coordinate information found for channels ['spkr', 'mic', 'vEOG', 'hEOG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  b2_raw = mne.io.read_raw_brainvision(b2_fpath,preload=True,verbose=False)
/tmp/ipykernel_38160/52083338.py:27: RuntimeWarning: Not setting positions of 4 misc channels found in montage:
['spkr', 'mic', 'vEOG', 'hEOG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creati

0 bad epochs dropped
Using data from preloaded Raw for 298 events and 641 original time points ...
    Rejecting  epoch based on EEG : ['C5']
    Rejecting  epoch based on EEG : ['F3', 'FT9', 'FC5', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FC3']
    Rejecting  epoch based on EEG : ['Oz', 'O2', 'PO4']
    Rejecting  epoch based on EEG : ['Fp1', 'Fz', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'Cz', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'FCz', 'C1', 'C2', 'FC4', 'FT8', 'F6', 'F2', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fz', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'Cz', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'FCz', 'C1', 'C5', 'CP3', 'C2', 'FC4', 'F6', 'F2', 'AF4', 'AF8']
5 bad epochs dropped
Interpolating TP9
Interpolating FT7
Creating RawArray with float64 data, n_channels=64, n_times=432083
    Range : 0 ... 432082 =      0.000 ...  3375.425 secs
Ready.
Interpolating bad channels
    Automatic

    Rejecting  epoch based on EEG : ['FT9']
    Rejecting  epoch based on EEG : ['FT9']
    Rejecting  epoch based on EEG : ['FT9']
    Rejecting  epoch based on EEG : ['FC2']
    Rejecting  epoch based on EEG : ['FT9']
    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EEG : ['FT9', 'F4']
15 bad epochs dropped
Using data from preloaded Raw for 150 events and 641 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 150 events and 641 original time points ...
    Rejecting  epoch based on EEG : ['FT9']
    Rejecting  epoch based on EEG : ['FT9']
    Rejecting  epoch based on EEG : ['FT9']
    Rejecting  epoch based on EEG : ['FT9']
    Rejecting  epoch based on EEG : ['FT9']
    Rejecting  epoch based on EEG : ['FT9']
    Rejecting  epoch based on EEG : ['FT9']
7 bad epochs dropped
Using data from preloaded Raw for 300 events and 641 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 300 events and 641 origina

    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fz', 'F3', 'F7', 'FC1', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['FC4']
    Rejecting  epoch based on EEG : ['AF7']
    Rejecting  epoch based on EEG : ['AF7']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['FC6', 'C2']
    Rejecting  epoch based on EEG : ['FC6', 'Fp2', 'C2', 'AF4']
    Rejecting  epoch based on EEG : ['AF7']
    Rejecting  epoch based on EEG : ['AF7']
13 bad epochs dropped
Using data from preloaded Raw for 300 events and 641 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 300 events and 641 original time points ...
    Rejecting  epoch based on EEG : ['Fp1', 'Fz', 'F3', 'FC5', 'FC6', 'F8', 'Fp2', 'AF7', 'F1', 'F5', 'C1', 'C2', 'FC4', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 

In [6]:
# Extract windows of interest
N100_window = [0.08,0.15]
P200_window = [0.15,0.25]
t = np.linspace(tmin,tmax,num=spkr_resp['OP0001']['spkr']['all'].shape[2])
N100_inds = np.where((t>N100_window[0]) & (t<N100_window[1]))[0]
P200_inds = np.where((t>P200_window[0]) & (t<P200_window[1]))[0]
N100_spkr_peak_amplitude,N100_mic_peak_amplitude = dict(),dict()
P200_spkr_peak_amplitude,P200_mic_peak_amplitude = dict(),dict()
N100_spkr_peak_latency,N100_mic_peak_latency = dict(),dict()
P200_spkr_peak_latency,P200_mic_peak_latency = dict(),dict()
peak_to_peak_spkr,peak_to_peak_mic = dict(),dict()
for subj in subjs:
    # SPKR
    # Get the minimum value at each epoch
    N100_spkr_peak_amplitude[subj] = spkr_resp[subj]['spkr']['all'].mean(1)[:,N100_inds].min(1) # amplitude
    latency_idx = N100_inds[spkr_resp[subj]['spkr']['all'].mean(1)[:,N100_inds].argmin(1)] # latency
    N100_spkr_peak_latency[subj] = t[latency_idx]
    P200_spkr_peak_amplitude[subj] = spkr_resp[subj]['spkr']['all'].mean(1)[:,P200_inds].max(1) # amplitude
    latency_idx = P200_inds[spkr_resp[subj]['spkr']['all'].mean(1)[:,P200_inds].argmax(1)] # latency
    P200_spkr_peak_latency[subj] = t[latency_idx]
    peak_to_peak_spkr[subj] = np.abs(
        P200_spkr_peak_amplitude[subj] - N100_spkr_peak_amplitude[subj])
    # MIC
    N100_mic_peak_amplitude[subj] = mic_resp[subj]['mic']['all'].mean(1)[:,N100_inds].min(1) # amplitude
    latency_idx = N100_inds[mic_resp[subj]['mic']['all'].mean(1)[:,N100_inds].argmin(1)] # latency
    N100_mic_peak_latency[subj] = t[latency_idx]
    P200_mic_peak_amplitude[subj] = mic_resp[subj]['mic']['all'].mean(1)[:,P200_inds].max(1) # amplitude
    latency_idx = P200_inds[mic_resp[subj]['mic']['all'].mean(1)[:,P200_inds].argmax(1)] # latency
    P200_mic_peak_latency[subj] = t[latency_idx]
    peak_to_peak_mic[subj] = np.abs(
        P200_mic_peak_amplitude[subj] - N100_mic_peak_amplitude[subj])

In [7]:
N100_el_peak_amplitude,N100_sh_peak_amplitude = dict(),dict()
P200_el_peak_amplitude,P200_sh_peak_amplitude = dict(),dict()
N100_el_peak_latency,N100_sh_peak_latency = dict(),dict()
P200_el_peak_latency,P200_sh_peak_latency = dict(),dict()
peak_to_peak_el,peak_to_peak_sh = dict(),dict()
for subj in subjs:
    if subj != 'OP0015': # no sh
        # el
        N100_el_peak_amplitude[subj] = spkr_resp[subj]['spkr']['el'].mean(1)[:,N100_inds].min(1) # amplitude
        latency_idx = N100_inds[spkr_resp[subj]['spkr']['el'].mean(1)[:,N100_inds].argmin(1)] # latency
        N100_el_peak_latency[subj] = t[latency_idx]
        P200_el_peak_amplitude[subj] = spkr_resp[subj]['spkr']['el'].mean(1)[:,P200_inds].max(1) # amplitude
        latency_idx = P200_inds[spkr_resp[subj]['spkr']['el'].mean(1)[:,P200_inds].argmax(1)] # latency
        P200_el_peak_latency[subj] = t[latency_idx]
        peak_to_peak_el[subj] = np.abs(
            P200_el_peak_amplitude[subj] - N100_el_peak_amplitude[subj])
        # sh
        N100_sh_peak_amplitude[subj] = spkr_resp[subj]['spkr']['sh'].mean(1)[:,N100_inds].min(1) # amplitude
        latency_idx = N100_inds[spkr_resp[subj]['spkr']['sh'].mean(1)[:,N100_inds].argmin(1)] # latency
        N100_sh_peak_latency[subj] = t[latency_idx]
        P200_sh_peak_amplitude[subj] = spkr_resp[subj]['spkr']['sh'].mean(1)[:,P200_inds].max(1) # amplitude
        latency_idx = P200_inds[spkr_resp[subj]['spkr']['sh'].mean(1)[:,P200_inds].argmax(1)] # latency
        P200_sh_peak_latency[subj] = t[latency_idx]
        peak_to_peak_sh[subj] = np.abs(P200_sh_peak_amplitude[subj] - N100_sh_peak_amplitude[subj])

In [8]:
# write to CSV
csvheader = [['Subject','Cond','N100_amp','P200_amp','N100_latency','P200_latency','peak_to_peak']]
csv_fname = f'{git_path}stats/lme/csvs/perception_production.csv'
conditions = ['Perc','Prod']
for i,subj in enumerate(subjs):
    for condition in conditions:
        if condition == 'Perc':
            for i,trial in enumerate(N100_spkr_peak_amplitude[subj]):
                csvheader.append([subj,condition,
                                  trial,P200_spkr_peak_amplitude[subj][i],
                                  N100_spkr_peak_latency[subj][i],P200_spkr_peak_latency[subj][i],
                                  peak_to_peak_spkr[subj][i]])
        elif condition == 'Prod':
            for i,trial in enumerate(N100_mic_peak_amplitude[subj]):
                csvheader.append([subj,condition,
                                  trial,P200_mic_peak_amplitude[subj][i],
                                  N100_mic_peak_latency[subj][i],P200_mic_peak_latency[subj][i],
                                  peak_to_peak_mic[subj][i]])
with open(csv_fname,'w+') as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(csvheader)

csvheader = [['Subject','Cond','N100_amp','P200_amp','N100_latency','P200_latency','peak_to_peak']]
csv_fname = f'{git_path}stats/lme/csvs/predictable_unpredictable.csv'
conditions = ['Echo','Shuff']
for i,subj in enumerate(subjs):
    if subj != 'OP0015': # no sh
        for condition in conditions:
            if condition == 'Echo':
                for i,trial in enumerate(N100_el_peak_amplitude[subj]):
                    csvheader.append([subj,condition,
                                      trial,P200_el_peak_amplitude[subj][i],
                                      N100_el_peak_latency[subj][i],P200_el_peak_latency[subj][i],
                                      peak_to_peak_el[subj][i]])
            elif condition == 'Shuff':
                for i,trial in enumerate(N100_sh_peak_amplitude[subj]):
                    csvheader.append([subj,condition,
                                      trial,P200_sh_peak_amplitude[subj][i],
                                      N100_sh_peak_latency[subj][i],P200_sh_peak_latency[subj][i],
                                      peak_to_peak_sh[subj][i]])
with open(csv_fname,'w+') as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(csvheader)

## Load LEM data

In [9]:
models = ['model1','model1e','model2','model2e','model3','model3e','model4','model4e']
exclude = ['OP0001','OP0002','OP0020']
subjs = [s for s in subjs if s not in exclude]
tmin,tmax = -0.3,0.5
delays = np.arange(np.floor(tmin*128),np.ceil(tmax*128),dtype=int)
features = {model_number:strf.get_feats(model_number,extend_labels=True) for model_number in models}
n_feats = {model_number:len(features[model_number]) for model_number in models}

In [10]:
# Load STRF data from hdf5
wts, corrs, pvals, sig_wts, sig_corrs, alphas = dict(), dict(), dict(), dict(), dict(), dict()
for m in tqdm(models):
    wts[m], corrs[m], pvals[m], sig_wts[m], sig_corrs[m], alphas[m] = dict(), dict(), dict(), dict(), dict(), dict()
    for s in subjs:
        f = h5py.File(f'{h5_path}{m}/{s}_strf.hdf5', 'r')
        wts[m][s] = np.array(f.get("wts"))
        corrs[m][s] = np.array(f.get("corrs"))
        pvals[m][s] = np.array(f.get("pvals"))
        alphas[m][s] = np.array(f.get("alphas"))
        f.close()
    # Extract significant weights, corrs
    for s in subjs:
        nchans = wts[m][s].shape[2]
        sig_wts[m][s] = np.zeros((len(delays),n_feats[m],nchans))
        sig_corrs[m][s] = np.zeros((nchans))
        for i in np.arange(nchans):
            if pvals[m][s][i] < 0.01:
                sig_wts[m][s][i] = wts[m][s][i]
                sig_corrs[m][s][i] = corrs[m][s][i]

  0%|          | 0/8 [00:00<?, ?it/s]

In [11]:
xm, ym = 'model1', 'model2'
header = [['r','model','subject','channel']]
for s in subjs:
    ch_names = mne.io.read_raw_brainvision(
        f'{eeg_data_path}{s}/{s}_B1/{s}_B1_cca.vhdr',
        preload=False,verbose=False).info['ch_names']
    for i,c in enumerate(ch_names):
        header.append([corrs[xm][s][i],xm,s,c])
        header.append([corrs[ym][s][i],ym,s,c])
csv_fname = f'{git_path}stats/lme/csvs/{xm}_{ym}.csv'
with open(csv_fname,'w+') as f:
    csvWriter = csv.writer(f)
    csvWriter.writerows(header)